In [ ]:
### CricMatchSheet Analysis Project ###

In [3]:
# extracting the json files and downloading it to a directory
# Tools used : 
    # Selenium for webscraping
    # json for file operations
    # BeautifulSoup to parse the html data and for dynamic access

import pandas as pd
import requests
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import os
option = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=option)

url="https://cricsheet.org/matches/"
driver.get(url)
drvr = driver.page_source
soup = BeautifulSoup(drvr,"html.parser")
links = driver.find_elements(By.XPATH,"""//dt[contains(text(),"Test matches") or 
                                             contains(text(),"One-day internationals") or
                                             contains(text(),"T20 internationals") or
                                             contains(text(),"Indian Premier League")]
                                             /following-sibling::dd//a[contains(@href,"json.zip")]""")
for link in links:
    href = link.get_attribute("href")
    json_filename = href.split("/")[-1]
    op_dir = r"C:\Users\sathy\OneDrive\Desktop\Project\Cric-MatchsheetDataAnalysis"
    json_filepath = os.path.join(op_dir,json_filename)
    try:
        json_response = requests.get(href, timeout=30)
        json_response.raise_for_status()
        with open(json_filepath,"wb") as ft:
            ft.write(json_response.content)
            print(f"Downloaded:{json_filename}")
    except requests.exceptions.Timeout as error:
        print("Timeout")
    except requests.exceptions.RequestException as er:
        print(f"Error downloading {json_filename}: {er}")
    except OSError as err:
        print(f"Error writing file{json_filename}: {err}")


Downloaded:tests_json.zip
Downloaded:mdms_json.zip
Downloaded:odis_json.zip
Downloaded:odms_json.zip
Downloaded:t20s_json.zip
Downloaded:it20s_json.zip
Downloaded:apl_json.zip
Downloaded:bpl_json.zip
Downloaded:bbl_json.zip
Downloaded:bwt_json.zip
Downloaded:cpl_json.zip
Downloaded:cec_json.zip
Downloaded:cch_json.zip
Downloaded:ipo_json.zip
Downloaded:ipt_json.zip
Downloaded:ctc_json.zip
Downloaded:wod_json.zip
Downloaded:frb_json.zip
Downloaded:ipl_json.zip
Downloaded:ilt_json.zip
Downloaded:lpl_json.zip
Downloaded:mlc_json.zip
Downloaded:msl_json.zip
Downloaded:npl_json.zip
Downloaded:rlc_json.zip
Downloaded:psl_json.zip
Downloaded:pks_json.zip
Downloaded:rhf_json.zip
Downloaded:sat_json.zip
Downloaded:ssh_json.zip
Downloaded:sft_json.zip
Downloaded:ssm_json.zip
Downloaded:sma_json.zip
Downloaded:ntb_json.zip
Downloaded:blz_json.zip
Downloaded:hnd_json.zip
Downloaded:wbb_json.zip
Downloaded:wcl_json.zip
Downloaded:wsl_json.zip
Downloaded:wpl_json.zip
Downloaded:wtc_json.zip


In [5]:
# extracting the json files from zip files
# reading the json files and extracting the necessary information
# converting the data into a pandas DataFrame(dict datatype)
# writing the DataFrame to a csv file for data storage

# Tools used:
    # python's zipfile for zipfile extraction
    # pandas to create the dataframe
    # json to perform file operations

import zipfile as zf
import pandas as pd
import json
import os
from datetime import datetime

ip_dir = r"C:\Users\sathy\OneDrive\Desktop\Project\Cric-MatchsheetDataAnalysis"
detailed_data = {"Test": [], "ODI": [], "T20": [], "IPL": []} #Initialization to append the data respectively.

# mapping the prefix of the filenames to each requested match types #
mapping = {
    "tests_": "Test",
    "odis_": "ODI",
    "t20s_": "T20",
    "ipl_": "IPL"
}

for filename in os.listdir(ip_dir):
    if filename.endswith(".zip"):
        filepath = os.path.join(ip_dir, filename)
        target_match_type = None
        for prefix, match_type in mapping.items():
            if filename.lower().startswith(prefix.lower()):
                target_match_type = match_type
                print(target_match_type)
                break
        if target_match_type:
            print(f"Processing files in {filename} as {target_match_type} matches...")
            try:
                with zf.ZipFile(filepath, "r") as zip_ref:
                    for name in zip_ref.namelist():
                        if name.endswith(".json"):
                            try:
                                with zip_ref.open(name, "r") as json_file:
                                    content = json_file.read().decode("utf-8", errors="ignore")
                                    try:
                                        match_data = json.loads(content)
                                        match_info = match_data.get("info", {})
                                        teams = match_info.get("teams", [])

                                        match_details = {
                                            "match_type": match_info.get("match_type"),
                                            "season": match_info.get("season"),
                                            "city": match_info.get("city"),
                                            "venue": match_info.get("venue"),
                                            "toss_winner": match_info.get("toss", {}).get("winner"),
                                            "toss_decision": match_info.get("toss", {}).get("decision"),
                                            "winner": match_info.get("outcome", {}).get("winner", "draw"),
                                            "player_of_match": ", ".join(match_info.get("player_of_match", [])),
                                            "teams": ", ".join(teams),
                                        }

                                        for inning in match_data.get("innings", []):
                                            team = inning.get("team", "Unknown")
                                            for over in inning.get("overs", []):
                                                over_number = over.get("over")
                                                for delivery in over.get("deliveries", []):
                                                    delivery_data = {
                                                        "team": team,
                                                        "over": over_number,
                                                        "batter": delivery.get("batter"),
                                                        "bowler": delivery.get("bowler"),
                                                        "non_striker": delivery.get("non_striker"),
                                                        "runs_batter": delivery.get("runs", {}).get("batter", 0),
                                                        "runs_extras": delivery.get("runs", {}).get("extras", 0),
                                                        "runs_total": delivery.get("runs", {}).get("total", 0),
                                                        "wicket": delivery.get("wickets", [{}])[0].get("player_out", "None") if "wickets" in delivery else "None"
                                                    }
                                                    detailed_data[target_match_type].append({**match_details, **delivery_data}) # Appending the respective match_type to the correct list

                                    except json.JSONDecodeError as je:
                                        print(f" *** JSONDecodeError in {filename}/{name} ***: {je}")
                            except Exception as er:
                                print(f" error reading json {filename}/{name} : {er}")
            except zf.BadZipFile as err:
                print(f"Error reading zipfile {filepath}: {err}")
            except Exception as errr:
                print(f"Error processing the file {filepath}: {errr}")

# Create separate DataFrames for each match_type
# Convert the respective DataFrames to respective csv files for data storage and future use

test_df = pd.DataFrame(detailed_data.get("Test", []))
test_df.to_csv(r'C:\Users\sathy\OneDrive\Desktop\Project\Cric-MatchsheetDataAnalysis\test.csv',index=False)
odi_df = pd.DataFrame(detailed_data.get("ODI", []))
odi_df.to_csv(r'C:\Users\sathy\OneDrive\Desktop\Project\Cric-MatchsheetDataAnalysis\ODI.csv',index=False)
t20_df = pd.DataFrame(detailed_data.get("T20", []))
t20_df.to_csv(r'C:\Users\sathy\OneDrive\Desktop\Project\Cric-MatchsheetDataAnalysis\T20.csv',index=False)
ipl_df = pd.DataFrame(detailed_data.get("IPL", []))
ipl_df.to_csv(r'C:\Users\sathy\OneDrive\Desktop\Project\Cric-MatchsheetDataAnalysis\IPL.csv',index=False)

# Print the head of each DataFrame
if not test_df.empty:
    print("\n Test DataFrame:")
    print(test_df.head())

if not odi_df.empty:
    print("\n ODI DataFrame:")
    print(odi_df.head())

if not t20_df.empty:
    print("\n T20 DataFrame:")
    print(t20_df.head())

if not ipl_df.empty:
    print("\n IPL DataFrame:")
    print(ipl_df.head())

IPL
Processing files in ipl_json.zip as IPL matches...
ODI
Processing files in odis_json.zip as ODI matches...
T20
Processing files in t20s_json.zip as T20 matches...
Test
Processing files in tests_json.zip as Test matches...

 Test DataFrame:
  match_type   season  city                  venue toss_winner toss_decision  \
0       Test  2007/08  None  Sydney Cricket Ground   Australia           bat   
1       Test  2007/08  None  Sydney Cricket Ground   Australia           bat   
2       Test  2007/08  None  Sydney Cricket Ground   Australia           bat   
3       Test  2007/08  None  Sydney Cricket Ground   Australia           bat   
4       Test  2007/08  None  Sydney Cricket Ground   Australia           bat   

      winner player_of_match             teams       team  over     batter  \
0  Australia       A Symonds  Australia, India  Australia     0  PA Jaques   
1  Australia       A Symonds  Australia, India  Australia     0  PA Jaques   
2  Australia       A Symonds  Australia, 

In [7]:
# Data Storage - Table creation and data insertion
# Read the csv files for each match type(ODI, TEST, T20, IPL) and create separate tables respectively.
# Insert the data into the respective tables.

# Tools used:
    # pandas -> to read the csv files, convert it to dataframe and pre-process the data before insertion
    # MYSQL Connector -> to create tables and insert data

import pandas as pd
import mysql.connector
import os

# DATABASE Credentials #
DB_HOST = "localhost"  
DB_USER = "root"    
DB_PASSWORD = "893107"  
DB_NAME = "cricket"      

# Declaring the input file path to access the saved CSV files #
INPUT_DIR = r"C:\Users\sathy\OneDrive\Desktop\Project\Cric-MatchsheetDataAnalysis"

# Table Creation -> for each match type(TEST, ODI, T20, IPL) respectively.
def create_table(cursor, table_name):
    try:
        cursor.execute(f"""
        CREATE TABLE IF NOT EXISTS `{table_name}` (
            `match_type` VARCHAR(50),
            `season` VARCHAR(50),
            `city` VARCHAR(50),
            `venue` VARCHAR(255),
            `toss_winner` VARCHAR(100),
            `toss_decision` VARCHAR(50),
            `winner` VARCHAR(100),
            `player_of_match` TEXT,
            `teams` TEXT,
            `team` VARCHAR(100),
            `over` INT,
            `batter` VARCHAR(100),
            `bowler` VARCHAR(100),
            `non_striker` VARCHAR(100),
            `runs_batter` INT,
            `runs_extras` INT,
            `runs_total` INT,
            `wicket` VARCHAR(100)
        )
        """)
        print(f"Table '{table_name}' created or already exists.")
    except mysql.connector.Error as err:
        print(f"Error creating table '{table_name}': {err}")

# Data Insertion -> Inserting the relevant data into the tables.
def insert_data(cursor, table_name, df):
    """Inserts data from a pandas DataFrame into the specified MySQL table."""
    cols = ",".join([f"`{col}`" for col in df.columns])
    placeholders = ",".join(["%s"] * len(df.columns))
    sql = f"INSERT INTO `{table_name}` ({cols}) VALUES ({placeholders})"
    try:
        for i, row in df.iterrows():
            cursor.execute(sql, tuple(row))
        print(f"{df.shape[0]} records inserted into '{table_name}'.")
    except mysql.connector.Error as err:
        print(f"Error inserting data into '{table_name}': {err}")

if __name__ == "__main__":
    try:
        mydb = mysql.connector.connect(
            host=DB_HOST,
            user=DB_USER,
            password=DB_PASSWORD,
            database=DB_NAME
        )
        mycursor = mydb.cursor()

        # Create tables for each match type #
        create_table(mycursor, "test_matches")
        create_table(mycursor, "odi_matches")
        create_table(mycursor, "t20_matches")
        create_table(mycursor, "ipl_matches") 

        # Insert data from CSV files for each match type #
        csv_files = {
            "test": "test_matches",
            "odi": "odi_matches",
            "t20": "t20_matches",
            "ipl": "ipl_matches"
        }

        # Pre-process the data before inserting it to tables #
        for csv_prefix, table_name in csv_files.items():
            csv_filepath = os.path.join(INPUT_DIR, f"{csv_prefix}.csv")
            if os.path.exists(csv_filepath):
                try:
                    df = pd.read_csv(csv_filepath,dtype={"season":str},low_memory=False)
                    df = df.where(pd.notnull(df), None)
                    insert_data(mycursor, table_name, df)
                    mydb.commit() # Commit the changes after each successful insertion so data is saved.
                except pd.errors.EmptyDataError:
                    print(f"Warning: '{csv_filepath}' is empty, skipping insertion into '{table_name}'.")
                except FileNotFoundError:
                    print(f"Error: CSV file '{csv_filepath}' not found.")
            else:
                print(f"Warning: CSV file '{csv_filepath}' does not exist.")

        print("Database operations completed successfully.")

    except mysql.connector.Error as err:
        print(f"Error connecting to MySQL: {err}")

    finally:
        if 'mydb' in locals() and mydb.is_connected():
            mycursor.close()
            mydb.close()
            print("MySQL connection closed.")

Table 'test_matches' created or already exists.
Table 'odi_matches' created or already exists.
Table 't20_matches' created or already exists.
Table 'ipl_matches' created or already exists.
1672502 records inserted into 'test_matches'.
1564696 records inserted into 'odi_matches'.
942914 records inserted into 't20_matches'.
274379 records inserted into 'ipl_matches'.
Database operations completed successfully.
MySQL connection closed.
